This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [ ]:
#!micromamba install pandera pandera-hypotheses pandera-io pandera-geopandas -c conda-forge -y

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
from typing import Union
import pandas as pd
import geopandas as gpd
import numpy as np

from helpers.strapi import Strapi

from data_commons.loader import load_regions

from pipelines.output_schemas import (
    StablishmentStageSchema,
    FPLSchema,
    ProtectionLevelSchema,
    MPAsSchema,
    MPAsTableStatsSchema,
    HabitatsSchema,
)

In [30]:
eez_folder = Path("../data/eez_intermediate").absolute()
mpa_folder = Path("../data/mpa_intermediate").absolute()
mpaatlas_folder = Path("../data/mpaatlas_intermediate").absolute()
protectedseas_folder = Path("../data/protectedseas_intermediate").absolute()
habitats_folder = Path("../data/habitats_intermediate").absolute()


location_code = pd.read_csv(eez_folder.joinpath("locations_code.csv"))

In [5]:
habitats_intermediate = pd.read_csv(habitats_folder.joinpath("habitats4.csv"), keep_default_na=False)

In [58]:

mpa_intermediate = gpd.read_file(mpa_folder.joinpath("mpa_intermediate", "mpa_intermediate.shp"))

In [59]:
protectedseas_intermediate = gpd.read_file(
    protectedseas_folder.joinpath("protectedseas_intermediate", "protectedseas_intermediate.shp")
)

In [32]:
mpaatlas_intermediate = gpd.read_file(
    mpaatlas_folder.joinpath("mpaatlas_intermediate", "mpaatlas_intermediate.shp").as_posix()
)

In [33]:
def separate_parent_iso(df: pd.DataFrame, iso_column="location_i", separator=";") -> pd.DataFrame:
    df[iso_column] = (
        df[iso_column].str.replace(" ", "").str.replace(":", separator).str.split(separator)
    )
    return df.explode(iso_column)


def calculate_area(df: pd.DataFrame, output_area_column="area_km2") -> pd.DataFrame:
    df[output_area_column] = (df.to_crs("ESRI:54009")["geometry"].area / 10**6).round(2)
    return df


def calculate_global_area(
    df: pd.DataFrame, gby_col: list, output_area_column="area_km2", iso_column="location_i"
) -> pd.DataFrame:
    global_area = (
        df.groupby([*gby_col])
        .agg({output_area_column: "sum"})
        .reset_index()
        .assign(**{iso_column: "GLOB"})
    )
    return pd.concat([global_area, df], ignore_index=True)


def add_region_iso(df: pd.DataFrame, iso_column) -> pd.DataFrame:
    regions = load_regions()

    def find_region_iso(iso: str) -> Union[str, None]:
        filtered_regions = list(filter(lambda x: iso in x["country_iso_3s"], regions.get("data")))
        return filtered_regions[0]["region_iso"] if len(filtered_regions) > 0 else None

    return df.assign(region=lambda row: row[iso_column].apply(find_region_iso))

def mpaatlas_filter_stablishment(df: pd.DataFrame) -> pd.DataFrame:
    return df[(df["establishm"].isin(["actively managed", "implemented"]))].copy()

def mpaatlas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "location_i"})
    )

    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "location_i"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def protectedseas_calculation(df: pd.DataFrame, gby_col: list) -> pd.DataFrame:
    regions = (
        df.groupby([*gby_col, "region"])
        .agg({"area_km2": "sum"})
        .reset_index()
        .rename(columns={"region": "iso"})
    )
    return pd.concat(
        [
            regions,
            df.groupby([*gby_col, "iso"]).agg({"area_km2": "sum"}).reset_index(),
        ],
        ignore_index=True,
    )


def batch_export(df: pd.DataFrame, batch_size: int, schema: object, folder: Path, filename: str):
    prev = 0
    for idx, size in enumerate(range(batch_size, len(df.index) + batch_size, batch_size)):
        schema(df[(df.index > prev) & (df.index < size)]).to_csv( # type: ignore
            folder.joinpath(f"{filename}_{idx}.csv"),
            index=True,
            encoding="utf-8",
        )
        prev = size


def fix_monaco(df: pd.DataFrame, iso_column="location_i", area_column="area_km2") -> pd.DataFrame:
    df.loc[df[iso_column] == "MCO", area_column] = 288
    return df


def set_area(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(
        area_km2=df[["area_km2_x", "area_km2_y", "area"]].bfill(axis=1)[["area_km2_x"]]
    )


def output(
    df: pd.DataFrame, iso_column: str, rep_d: dict, rename: dict, drop_cols: list
) -> pd.DataFrame:
    if iso_column:
        locations_code = pd.read_csv(
            eez_folder.joinpath("locations_code.csv"), keep_default_na=False
        )
        df = df.join(locations_code.set_index("code"), on=iso_column, how="left")
    return (
        df.replace(rep_d)
        .rename(columns=rename)
        .drop(columns=drop_cols)
        .assign(
            id=df.index + 1,
        )
        .set_index("id")
    )


def filter_location(df: pd.DataFrame) -> pd.DataFrame:
    return df[~df.location.isna()]

### habitats

In [14]:
habitats_intermediate

,location_id,protected_area,total_area,habitat_name,year
0,AGO,0.0,3.395671,cold-water corals,2024
1,ALB,0.0,5.986479,cold-water corals,2024
2,ARG,6.98422602063557,61.826344,cold-water corals,2024
3,ATA,5.41915117560982,18.889068,cold-water corals,2024
4,ATG,0.0,0.997747,cold-water corals,2024
...,...,...,...,...,...
611,AS,21277.219999999998,74292.673146,mangroves,2020
612,EU,732.14375,1246.189677,mangroves,2020
613,NA,2097.74,2415.418557,mangroves,2020
614,SA,27151.739999999998,39893.444608,mangroves,2020


In [62]:
habitat_stats.replace({"protectedArea":{"": None}}).astype({"protectedArea": "float64"}).info()

<class 'pandas.core.frame.DataFrame'>
Index: 616 entries, 1 to 616
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   protectedArea  570 non-null    float64
 1   totalArea      616 non-null    float64
 2   habitat        616 non-null    int64  
 3   year           616 non-null    int64  
 4   location       616 non-null    int64  
dtypes: float64(2), int64(3)
memory usage: 28.9 KB


In [64]:
habitat_stats = habitats_intermediate.pipe(
    output,
    iso_column="location_id",
    rep_d={
        "habitat_name": {
            "saltmarshes": 1,
            "seagrasses": 2,
            "warm-water corals": 3,
            "cold-water corals": 4,
            "mangroves": 5,
            "seamounts": 6,
        },
        "protected_area": {"": 0},
    },
    rename={
        "protected_area": "protectedArea",
        "total_area": "totalArea",
        "habitat_name": "habitat",
    },
    drop_cols=["location_id"],
)
HabitatsSchema(habitat_stats).to_csv(habitats_folder.joinpath("habitats_stats.csv"), index=True)

### mpa atlas

In [34]:
mpa_atlas_table = (
    mpaatlas_intermediate.pipe(calculate_area)
)

In [35]:
mpa_atlas_table

,wdpa_id,designatio,name,location_i,establishm,protection,protecti_1,year,geometry,area_km2
0,NaN,Marine Sanctuary,Jorge Orlando Mera,DOM,designated,unknown/to be determined,less protected or unknown,2024,"POLYGON ((-72.80605 16.82674, -71.73463 17.773...",55110.25
1,NaN,Marine Mammal Sanctuary,Bancos de La Plata y La Navidad [Expansion],DOM,designated,unknown/to be determined,less protected or unknown,2024,"POLYGON ((-67.37611 20.53249, -67.39195 20.478...",31306.65
2,NaN,Exclusive Economic Zone,Tristan da Cunha [Seamount Fishing Zones],GBR,actively managed,full,fully or highly protected,2021,"MULTIPOLYGON (((-8.56250 -40.26546, -8.57083 -...",11300.78
3,555720256,Exclusive Economic Zone,Tristan da Cunha [Inshore Fishing Zones],GBR,actively managed,light,less protected or unknown,2021,"MULTIPOLYGON (((-9.01170 -40.24569, -9.00301 -...",59082.84
4,NaN,Marine Park,Macquarie Island [National Park Zone],AUS,implemented,high,fully or highly protected,2023,"POLYGON ((164.64502 -54.96694, 164.63141 -54.9...",387947.47
...,...,...,...,...,...,...,...,...,...,...
805,555629385_A,National Park,Revillagigedo - Zona Núcleo/Uso Restringido In...,MEX,actively managed,full,fully or highly protected,2019,"POLYGON ((-110.07809 20.00863, -110.07809 17.6...",148493.24
806,555624307_A,Biosphere Reserve,Pacífico Mexicano Profundo [Buffer Zone],MEX,implemented,incompatible,less protected or unknown,2018,"MULTIPOLYGON (((-94.70558 15.19730, -92.76017 ...",315766.53
807,555624307_B,Biosphere Reserve,Pacífico Mexicano Profundo [Core Zone],MEX,implemented,incompatible,less protected or unknown,2018,"MULTIPOLYGON (((-93.29993 13.95000, -93.29993 ...",122356.76
808,555624306_A,Biosphere Reserve,Caribe Mexicano Buffer Zone,MEX,actively managed,incompatible,less protected or unknown,2016,"MULTIPOLYGON (((-86.06299 18.48397, -86.05480 ...",38809.04


In [10]:
## DeprecationWarning indicator not used
test = (
    mpa_atlas_table.pipe(calculate_global_area, gby_col=["establishm"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["establishm"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "establishm": {
                "actively managed": 4,
                "implemented": 6,
                "designated": 5,
                "proposed or committed": 3,
                "unknown": 1,
            }
        },
        rename={"establishm": "mpaa_establishment_stage", "area_km2": "area"},
        drop_cols=["location_i"],
    )
).assign(year=2023, protection_status=1)

StablishmentStageSchema(test[~test.location.isna()]).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_stablishment.csv"), index=True
)

In [37]:
test2 = (
    mpa_atlas_table
    .pipe(mpaatlas_filter_stablishment)
    .pipe(calculate_global_area, gby_col=["protecti_1"], iso_column="location_i")
    .pipe(separate_parent_iso)
    .replace(
        {
            "location_i": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="location_i")
    .pipe(mpaatlas_calculation, gby_col=["protecti_1"])
    .pipe(fix_monaco, iso_column="location_i", area_column="area_km2")
    .pipe(
        output,
        iso_column="location_i",
        rep_d={
            "protecti_1": {
                "fully or highly protected": 1,
                "less protected or unknown": 2,
            }
        },
        rename={"protecti_1": "mpaa_protection_level", "area_km2": "area"},
        drop_cols=[],
    )
)
ProtectionLevelSchema(test2[~test2.location.isna()].assign(year=2023)).to_csv(
    mpaatlas_folder.joinpath("mpaatlas_protection_level.csv"), index=True
)

In [52]:
mpa_atlas_table[mpa_atlas_table.location_i.isin(["ECU"])].groupby(["establishm", "protection"]).agg(
    {"area_km2": "sum", "wdpa_id": "count"}
)

area_km2  wdpa_id
establishm       protection                                  
actively managed full                       45651.74        2
                 high                       94008.36        2
designated       unknown/to be determined  146962.03        1
implemented      full                       30273.45        1
                 incompatible               29783.22        1

In [42]:
mpa_atlas_table.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   wdpa_id     719 non-null    object  
 1   designatio  810 non-null    object  
 2   name        810 non-null    object  
 3   location_i  809 non-null    object  
 4   establishm  810 non-null    object  
 5   protection  810 non-null    object  
 6   protecti_1  810 non-null    object  
 7   year        525 non-null    object  
 8   geometry    808 non-null    geometry
 9   area_km2    808 non-null    float64 
dtypes: float64(1), geometry(1), object(8)
memory usage: 63.4+ KB


In [109]:
mpa_atlas_table[
    mpa_atlas_table.location_i.isin([["USA"]]) & mpa_atlas_table.protection.isin(["full", "high"])
]

,wdpa_id,designatio,name,location_i,establishm,protection,protecti_1,year,geometry,area_km2
357,NaN,Particularly Sensitive Sea Area,Papahanaumokuakea Particularly Sensitive Sea Area,[USA],implemented,high,fully or highly protected,2008,"POLYGON ((-161.92813 22.22347, -161.93001 22.2...",363657.07
388,555586884,Marine Life Conservation District,Molokini Shoal,[USA],actively managed,high,fully or highly protected,1977,"POLYGON ((-156.49247 20.63389, -156.49239 20.6...",0.36
389,21038,Natural Area Reserve,Ahihi-Kinau,[USA],actively managed,full,fully or highly protected,1973,"POLYGON ((-156.43912 20.62006, -156.43903 20.6...",8.40
390,555586717,State Marine Reserve,Año Nuevo,[USA],actively managed,full,fully or highly protected,2016,"POLYGON ((-122.36283 37.16545, -122.36190 37.1...",28.92
391,555586898,State Marine Recreational Management Area,Estero de San Antonio,[USA],actively managed,full,fully or highly protected,2010,"POLYGON ((-122.97100 38.27483, -122.97093 38.2...",0.19
...,...,...,...,...,...,...,...,...,...,...
793,12438,Nature Reserve,Isla de Mona Natural Reserve (No-Take Zone),[USA],actively managed,full,fully or highly protected,1986,"MULTIPOLYGON (((-67.90476 18.16181, -67.90470 ...",87.45
798,NaN,Marine Life Conservation District,Honolua-Mokuleia Bay,[USA],implemented,high,fully or highly protected,NaN,"MULTIPOLYGON (((-156.64162 21.01404, -156.6416...",0.18
799,NaN,Marine Life Conservation District,Hanauma Bay,[USA],actively managed,full,fully or highly protected,NaN,"POLYGON ((-157.69182 21.26368, -157.69181 21.2...",0.41
800,555586885,Island Reserve,Kahoolawe - Zone B,[USA],actively managed,high,fully or highly protected,1994,"POLYGON ((-156.52584 20.61583, -156.50501 20.5...",202.95


In [108]:
mpa_atlas_table[mpa_atlas_table.wdpa_id.isin(["555548204"])]

,wdpa_id,designatio,name,location_i,establishm,protection,protecti_1,year,geometry,area_km2
359,555548204,Marine Park,Great Barrier Reef Coast - Habitat Protection ...,[AUS],actively managed,light,less protected or unknown,NaN,"MULTIPOLYGON (((152.00901 -24.43617, 152.00796...",17313.80
360,555548204,Marine Park,Great Barrier Reef Coast - Conservation Park Zone,[AUS],actively managed,light,less protected or unknown,NaN,"MULTIPOLYGON (((151.94032 -24.23285, 151.94542...",4069.01
361,555548204,Marine Park,Great Barrier Reef Coast - Buffer Zone,[AUS],actively managed,high,fully or highly protected,NaN,"MULTIPOLYGON (((145.53308 -14.80205, 145.53308...",39.33
362,555548204,Marine Park,Great Barrier Reef Coast - Scientific Research...,[AUS],actively managed,high,fully or highly protected,NaN,"MULTIPOLYGON (((152.10886 -23.47565, 152.10886...",118.21
363,555548204,Marine Park,Great Barrier Reef Coast - Marine National Par...,[AUS],actively managed,high,fully or highly protected,NaN,"MULTIPOLYGON (((152.11465 -24.49841, 152.10000...",15765.97
364,555548204,Marine Park,Great Barrier Reef Coast - Preservation Zone,[AUS],actively managed,full,fully or highly protected,NaN,"MULTIPOLYGON (((150.95129 -23.33599, 150.95129...",295.54
515,555548204,Marine Park,Great Barrier Reef Coast - General Use Zone,[AUS],actively managed,incompatible,less protected or unknown,2004,"MULTIPOLYGON (((152.00847 -24.42670, 152.00897...",25297.97
518,555548204,Marine Park,Great Barrier Reef Coast - Estuarine Conservat...,[AUS],actively managed,light,less protected or unknown,2004,"MULTIPOLYGON (((146.10777 -17.60846, 146.10777...",118.36


In [106]:
test4_final[test4_final.wdpaid.str.contains("555548204")]

,WDPAID,wdpaid,protection_status,name,area,year,location
id,,,,,,,
10947,555548204,555548204_C,1,Great Barrier Reef Coast,3958.947309,2004,16.0
10948,555548204,555548204_B,1,Great Barrier Reef Coast,15689.429991,2004,16.0
10949,555548204,555548204_A,1,Great Barrier Reef Coast,405.725452,2004,16.0
10950,555548204,555548204_D,1,Great Barrier Reef Coast,41162.824965,2004,16.0


In [46]:
test2

,mpaa_protection_level,location_i,area,location
id,,,,
1,1,AF,30877.79,3
2,1,AS,2338083.01,4
3,1,EU,2678292.73,6
4,1,NA,3442537.50,7
5,1,SA,520828.06,8
...,...,...,...,...
60,2,NZL,1166727.62,118
61,2,PHL,1.85,123
62,2,PLW,117435.05,124


In [54]:
(test2[test2.location_i.isin(["GLOB"])]["area"] / 361000000) * 100

id
22    2.984289
50    3.002278
Name: area, dtype: float64

In [57]:
(test2[test2.location_i.isin(["GLOB"])]["area"] / 365098561) * 100

id
22    2.950788
50    2.968575
Name: area, dtype: float64

In [56]:
(20810618/5.7)*100

365098561.4035088

### protected seas

In [60]:
protected_seas_table = (
    protectedseas_intermediate.pipe(calculate_area)
)

In [61]:
test3 = (
    protected_seas_table.pipe(calculate_global_area, gby_col=["FPS_cat"], iso_column="iso")
    .pipe(separate_parent_iso, iso_column="iso")
    .replace(
        {
            "iso": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
    .pipe(add_region_iso, iso_column="iso")
    .pipe(protectedseas_calculation, gby_col=["FPS_cat"])
    .pipe(fix_monaco, iso_column="iso", area_column="area_km2")
    .pipe(
        output,
        iso_column="iso",
        rep_d={
            "FPS_cat": {
                "highly": 1,
                "moderately": 2,
                "less": 3,
            }
        },
        rename={"FPS_cat": "fishing_protection_level", "area_km2": "area"},
        drop_cols=["iso"],
    )
)

FPLSchema(test3).to_csv(protectedseas_folder.joinpath("lfp.csv"), index=True)

### Mpa protected planet

In [62]:
test4 = (
    mpa_intermediate.fillna(0)
    .replace(
        {
            "PARENT_ISO": {
                "COK": "NZL",
                "IOT": "GBR",
                "NIU": "NZL",
                "SHN": "GBR",
                "SJM": "NOR",
                "UMI": "USA",
                "NCL": "FRA",
            }
        }
    )
)

In [104]:
test4_final = test4.pipe(
    output,
    iso_column="PARENT_ISO",
    rep_d={
        "STATUS": {
            "Adopted": 4,
            "implemented": 6,
            "Established": 6,
            "Designated": 5,
            "Proposed": 3,
            "Inscribed": 3,
            "unknown": 1,
        },
        "PA_DEF": {"0": 2, "1": 1},
        "STATUS_YR": {0: pd.NA},
    },
    rename={
        "PARENT_ISO": "iso",
        "PA_DEF": "protection_status",
        "GIS_M_AREA": "area",
        "STATUS_YR": "year",
        "WDPA_PID": "wdpaid",
        "NAME": "name",
    },
    drop_cols=["geometry", "iso", "STATUS"],  # "WDPAID",
).astype({"year": "Int64"})

prev = 0
for idx, size in enumerate(range(5000, len(test4_final.index) + 5000, 5000)):
    MPAsSchema(test4_final[(test4_final.index > prev) & (test4_final.index < size)]).to_csv(
        mpa_folder.joinpath(f"mpa_{idx}.csv"),
        index=True,
        encoding="utf-8",
    )
    prev = size

In [93]:
test4.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18732 entries, 0 to 18731
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   WDPAID      18732 non-null  int64   
 1   WDPA_PID    18732 non-null  object  
 2   PA_DEF      18732 non-null  object  
 3   NAME        18732 non-null  object  
 4   GIS_M_AREA  18732 non-null  float64 
 5   STATUS      18732 non-null  object  
 6   STATUS_YR   18732 non-null  int64   
 7   PARENT_ISO  18732 non-null  object  
 8   geometry    18732 non-null  geometry
dtypes: float64(1), geometry(1), int64(2), object(5)
memory usage: 1.3+ MB


### Merged table stats

In [64]:
mpa_atlas_merge = mpa_atlas_table.pipe(separate_parent_iso, iso_column="location_i").replace(
    {
        "location_i": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

protectedseas_merge = protected_seas_table.pipe(separate_parent_iso, iso_column="iso").replace(
    {
        "iso": {
            "COK": "NZL",
            "IOT": "GBR",
            "NIU": "NZL",
            "SHN": "GBR",
            "SJM": "NOR",
            "UMI": "USA",
            "NCL": "FRA",
        }
    }
)

In [95]:
Final = (
    test4_final
    .assign(mpa=test4_final.index)
    .merge(
        mpa_atlas_merge[["establishm", "wdpa_id", "protection", "area_km2"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .merge(
        protectedseas_merge[["site_id", "wdpa_id", "area_km2", "FPS_cat"]],
        left_on="wdpaid",
        right_on="wdpa_id",
        how="left",
    )
    .pipe(set_area)
    .pipe(filter_location)
    .drop_duplicates()
    .reset_index(drop=True)
)

In [66]:
Final[["area_km2_x", "area_km2_y", "area"]]

,area_km2_x,area_km2_y,area
0,NaN,14.67,14.636135
1,NaN,3.23,3.845623
2,NaN,11.11,9.989930
3,NaN,NaN,353.837622
4,NaN,6616.77,2270.594697
...,...,...,...
18888,NaN,NaN,0.000000
18889,NaN,NaN,0.000000
18890,NaN,NaN,0.000000
18891,NaN,NaN,0.000000


In [67]:
Final[["area_km2_x", "area_km2_y", "area"]].bfill(axis=1)

,area_km2_x,area_km2_y,area
0,14.670000,14.670000,14.636135
1,3.230000,3.230000,3.845623
2,11.110000,11.110000,9.989930
3,353.837622,353.837622,353.837622
4,6616.770000,6616.770000,2270.594697
...,...,...,...
18888,0.000000,0.000000,0.000000
18889,0.000000,0.000000,0.000000
18890,0.000000,0.000000,0.000000
18891,0.000000,0.000000,0.000000


In [68]:
Final[Final.location==88]


,wdpaid,protection_status,name,area,year,location,mpa,establishm,wdpa_id_x,protection,area_km2_x,site_id,wdpa_id_y,area_km2_y,FPS_cat,area_km2
5625,309888,1,Phoenix Islands Protected Area,400043.739458,2006,88.0,5554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400043.739458
6259,555624127,1,Nooto-North Tarawa,0.000000,2013,88.0,6182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
8245,555512002,1,Phoenix Islands Protected Area,408145.610180,2010,88.0,8152,designated,555512002,unknown/to be determined,408258.13,NaN,NaN,NaN,NaN,408258.130000
12572,10753,1,Vostock Island,0.000000,1979,88.0,12437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12573,4257,1,Cook Islet Closed Area (Kiritimati WS),0.000000,1975,88.0,12438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12574,4258,1,Motu Tabu Islet Closed Area (Kiritimati),0.000000,1975,88.0,12439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12575,4259,1,Motu Upua Closed Area,0.000000,1975,88.0,12440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12576,4261,1,Ngaontetaake Islet Closed Area (Kiritimati),0.000000,1979,88.0,12441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12577,555547607,1,North-west Point Closed Area (Kiritimati),0.000000,1975,88.0,12442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
12578,4253,1,Kiritimati Atoll (Christmas Island),0.000000,1999,88.0,12443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [69]:
Final.to_csv("../data/merge_country_table_qa.csv", index=True)

In [70]:
Final.protection.unique()

array([nan, 'unknown/to be determined', 'full', 'light', 'incompatible',
       'high', 'minimal', 'unknown'], dtype=object)

In [71]:
Final.FPS_cat.unique()

array(['highly', 'less', nan, 'moderately'], dtype=object)

In [72]:
Final_output = Final.pipe(
    output,
    iso_column=None,
    rep_d={
        "protection": {
            "full": 3,
            "light": 4,
            "incompatible": 5,
            "high": 6,
            "minimal": 7,
            "unknown": 8,
            "unknown/to be determined": 8,
        },
        "FPS_cat": {
            "highly": 1,
            "moderately": 2,
            "less": 3,
        },
        "establishm": {
            "actively managed": 4,
            "implemented": 6,
            "designated": 5,
            "proposed or committed": 3,
        },
    },
    rename={
        "establishm": "mpaa_establishment_stage",
        "protection": "mpaa_protection_level",
        "FPS_cat": "fishing_protection_level",
    },
    drop_cols=[
        "wdpaid",
        "wdpa_id_x",
        "wdpa_id_y",
        "area_km2_x",
        "area_km2_y",
        "protection_status",
        "name",
        "site_id",
        "year",
        "area",
    ],
).rename(columns={"area_km2": "area"})

#

In [73]:
batch_export(Final_output, 5000, MPAsTableStatsSchema, mpa_folder, "mpa_join_mpatlas_prot")

In [74]:
Final_output

,location,mpa,mpaa_establishment_stage,mpaa_protection_level,fishing_protection_level,area
id,,,,,,
1,15.0,1,NaN,NaN,1.0,14.670000
2,15.0,2,NaN,NaN,3.0,3.230000
3,27.0,3,NaN,NaN,1.0,11.110000
4,26.0,4,NaN,NaN,NaN,353.837622
5,26.0,5,NaN,NaN,1.0,6616.770000
...,...,...,...,...,...,...
18889,22.0,18728,NaN,NaN,NaN,0.000000
18890,22.0,18729,NaN,NaN,NaN,0.000000
18891,22.0,18730,NaN,NaN,NaN,0.000000


In [75]:
protectedseas_merge.wdpa_id.unique().shape

(10786,)

In [78]:
protectedseas_merge.wdpa_id.str.replace(' ','').str.split(';').explode().unique().shape

(11984,)

In [ ]:
# test_Final["area"] = test_Final.area_km2_y - test_Final.area_km2_x
# test_Final[(test_Final.area_km2_1 != 0) & (test_Final.area_km2_1.notna())]